# 1. 연합뉴스

# 1.(1) 검색어: '윤석열'

In [29]:
# 첫페이지
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성

# 연합뉴스 사이트 접속
url_Y = "https://www.yna.co.kr/"
driver.get(url_Y)

try:
    # 돋보기
    search_icon =  WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/header/div[1]/span[3]/button[2]/i[1]'))
    ) 
    search_icon.click()
    
    # 검색창 대기
    search_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="liveSrch01"]'))
    )
    # 검색어 입력
    search_input.send_keys("윤석열")
    search_input.send_keys(Keys.RETURN)  # 엔터키로 검색

    # 다음 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="search_detail_inbox"]/fieldset/div[1]/div[1]/label'))
    )
    
except Exception as e:
    print("검색창에 접근할 수 없습니다:", e) 
    driver.quit()
    exit()
    
# 검색 기간 설정
try:
    # 직접입력 누르기
    direct_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="perioddirect"]'))
    )
    direct_input.click()           

    #정확도 누르기
    accuracy = driver.find_element(By.XPATH, '//*[@id="srchAlign02"]')
    accuracy.click()
    
    # 검색 시작 날짜
    start_date = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="from"]'))
    )
    start_date.send_keys("20220215")
    # 검색 종료 날짜
    end_date = driver.find_element(By.XPATH, '//*[@id="to"]')
    end_date.click()
    end_date.send_keys("20220308") 

    # 검색 누르기
    search_icon = driver.find_element(By.XPATH, '//*[@id="search_click"]')
    search_icon.click()

    # 다음 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="article_list"]/div[2]/ul/li[1]/a/span[1]'))
    )
    
    # 더보기 누르기 
    more_articles = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="article_list"]/div[1]/span[2]/a'))
    )
    more_articles.click()
    
    
except Exception as e:
    print("검색 기간을 설정할 수 없습니다:", e)
    driver.quit()
    exit()
#-----------------------------------------------------------------------------------------------------
# 기사 데이터 크롤링
time.sleep(5)
titles, links, dates = [], [], [] # 제목, 링크, 작성일자를 저장할 리스트 생성
# 페이지 HTML 가져오기
page_source = driver.page_source
soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

# <div class="cts_atclst"> 내부의 모든 <li> 태그 찾기
div_tag = soup.find("div", class_="cts_atclst")
if div_tag:
    articles = div_tag.find_all("li")  # <li> 태그 리스트 반환
    for article in articles:
        try:
            # 제목 추출
            title_tag = article.find("span", class_="tt2")
            title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
            titles.append(title)

            # 링크 추출
            link_tag = article.find("a")
            link = link_tag["href"] if link_tag and link_tag.has_attr("href") else "링크 없음"
            links.append(link)

            # 작성일자 추출
            date_tag = article.find("span", class_="pbdt")
            date = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
            dates.append(date)

        except Exception as e:
            print(f"개별 기사 처리 중 오류 발생: {e}")
     
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df = df.drop_duplicates()
    df.to_csv("윤석열_연합뉴스_기사목록.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.


In [31]:
## 이거 반복
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성

# 연합뉴스 사이트 접속
for numb in range(2,51):
    url_Y = f"https://www.yna.co.kr/search/index?query=%EC%9C%A4%EC%84%9D%EC%97%B4&ctype=A&sort=weight&from=20220215&to=20220308&period=diy&page_no={numb}"
    driver.get(url_Y)

#-----------------------------------------------------------------------------------------------------
    # 기사 데이터 크롤링
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

    # <div class="cts_atclst"> 내부의 모든 <li> 태그 찾기
    div_tag = soup.find("div", class_="cts_atclst")
    if div_tag:
        articles = div_tag.find_all("li")  # <li> 태그 리스트 반환
        for article in articles:
            try:
                # 제목 추출
                title_tag = article.find("span", class_="tt2")
                title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
                titles.append(title)

                # 링크 추출
                link_tag = article.find("a")
                link = link_tag["href"] if link_tag and link_tag.has_attr("href") else "링크 없음"
                links.append(link)

                # 작성일자 추출
                date_tag = article.find("span", class_="pbdt")
                date = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
                dates.append(date)

            except Exception as e:
                print(f"개별 기사 처리 중 오류 발생: {e}")
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df.to_csv("윤석열_연합뉴스_기사목록.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
    driver.quit()

데이터가 CSV 파일로 저장되었습니다.


In [7]:
## 그냥 이거 하나만
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
titles, links, dates = [], [], [] # 제목, 링크, 작성일자를 저장할 리스트 생성

# 연합뉴스 사이트 접속
for numb in range(1,21):
    url_Y = f"https://www.yna.co.kr/search/index?query=%EC%9C%A4%EC%84%9D%EC%97%B4&ctype=A&sort=weight&from=20220215&to=20220308&period=diy&page_no={numb}"
    driver.get(url_Y)

#-----------------------------------------------------------------------------------------------------
    # 기사 데이터 크롤링
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

    # <div class="cts_atclst"> 내부의 모든 <li> 태그 찾기
    div_tag = soup.find("div", class_="cts_atclst")
    if div_tag:
        articles = div_tag.find_all("li")  # <li> 태그 리스트 반환
        for article in articles:
            try:
                # 제목 추출
                title_tag = article.find("span", class_="tt2")
                title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
                titles.append(title)

                # 링크 추출
                link_tag = article.find("a")
                link = link_tag["href"] if link_tag and link_tag.has_attr("href") else "링크 없음"
                links.append(link)

                # 작성일자 추출
                date_tag = article.find("span", class_="pbdt")
                date = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
                dates.append(date)

            except Exception as e:
                print(f"개별 기사 처리 중 오류 발생: {e}")
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df.to_csv("윤석열_연합뉴스_기사목록200.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.


# 1.(2) 검색어: '이재명'

In [39]:
# 첫페이지
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성

# 연합뉴스 사이트 접속
url_Y = "https://www.yna.co.kr/"
driver.get(url_Y)

try:
    # 돋보기
    search_icon =  WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/header/div[1]/span[3]/button[2]/i[1]'))
    ) 
    search_icon.click()
    
    # 검색창 대기
    search_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="liveSrch01"]'))
    )
    # 검색어 입력
    search_input.send_keys("이재명")
    search_input.send_keys(Keys.RETURN)  # 엔터키로 검색

    # 다음 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="search_detail_inbox"]/fieldset/div[1]/div[1]/label'))
    )
    
except Exception as e:
    print("검색창에 접근할 수 없습니다:", e) 
    driver.quit()
    exit()
    
# 검색 기간 설정
try:
    # 직접입력 누르기
    direct_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="perioddirect"]'))
    )
    direct_input.click()           

    #정확도 누르기
    accuracy = driver.find_element(By.XPATH, '//*[@id="srchAlign02"]')
    accuracy.click()
    
    # 검색 시작 날짜
    start_date = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="from"]'))
    )
    start_date.send_keys("20220215")
    # 검색 종료 날짜
    end_date = driver.find_element(By.XPATH, '//*[@id="to"]')
    end_date.click()
    end_date.send_keys("20220308") 

    # 검색 누르기
    search_icon = driver.find_element(By.XPATH, '//*[@id="search_click"]')
    search_icon.click()

    # 다음 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="article_list"]/div[2]/ul/li[1]/a/span[1]'))
    )
    
    # 더보기 누르기 
    more_articles = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="article_list"]/div[1]/span[2]/a'))
    )
    more_articles.click()
    
    
except Exception as e:
    print("검색 기간을 설정할 수 없습니다:", e)
    driver.quit()
    exit()
#-----------------------------------------------------------------------------------------------------
# 기사 데이터 크롤링
time.sleep(5)
titles, links, dates = [], [], [] # 제목, 링크, 작성일자를 저장할 리스트 생성
# 페이지 HTML 가져오기
page_source = driver.page_source
soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

# <div class="cts_atclst"> 내부의 모든 <li> 태그 찾기
div_tag = soup.find("div", class_="cts_atclst")
if div_tag:
    articles = div_tag.find_all("li")  # <li> 태그 리스트 반환
    for article in articles:
        try:
            # 제목 추출
            title_tag = article.find("span", class_="tt2")
            title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
            titles.append(title)

            # 링크 추출
            link_tag = article.find("a")
            link = link_tag["href"] if link_tag and link_tag.has_attr("href") else "링크 없음"
            links.append(link)

            # 작성일자 추출
            date_tag = article.find("span", class_="pbdt")
            date = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
            dates.append(date)

        except Exception as e:
            print(f"개별 기사 처리 중 오류 발생: {e}")
     
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df = df.drop_duplicates()
    df.to_csv("이재명_연합뉴스_기사목록.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.


In [41]:
## 이거 반복
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성

# 연합뉴스 사이트 접속
for numb in range(2,51):
    url_Y = f"https://www.yna.co.kr/search/index?query=%EC%9D%B4%EC%9E%AC%EB%AA%85&ctype=A&sort=weight&from=20220215&to=20220308&period=diy&page_no={numb}"
    driver.get(url_Y)

#-----------------------------------------------------------------------------------------------------
    # 기사 데이터 크롤링
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

    # <div class="cts_atclst"> 내부의 모든 <li> 태그 찾기
    div_tag = soup.find("div", class_="cts_atclst")
    if div_tag:
        articles = div_tag.find_all("li")  # <li> 태그 리스트 반환
        for article in articles:
            try:
                # 제목 추출
                title_tag = article.find("span", class_="tt2")
                title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
                titles.append(title)

                # 링크 추출
                link_tag = article.find("a")
                link = link_tag["href"] if link_tag and link_tag.has_attr("href") else "링크 없음"
                links.append(link)

                # 작성일자 추출
                date_tag = article.find("span", class_="pbdt")
                date = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
                dates.append(date)

            except Exception as e:
                print(f"개별 기사 처리 중 오류 발생: {e}")
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df.to_csv("이재명_연합뉴스_기사목록.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
    driver.quit()

데이터가 CSV 파일로 저장되었습니다.


In [9]:
## 이거 하나만
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
titles, links, dates = [], [], [] # 제목, 링크, 작성일자를 저장할 리스트 생성

# 연합뉴스 사이트 접속
for numb in range(1,21):
    url_Y = f"https://www.yna.co.kr/search/index?query=%EC%9D%B4%EC%9E%AC%EB%AA%85&ctype=A&sort=weight&from=20220215&to=20220308&period=diy&page_no={numb}"
    driver.get(url_Y)

#-----------------------------------------------------------------------------------------------------
    # 기사 데이터 크롤링
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

    # <div class="cts_atclst"> 내부의 모든 <li> 태그 찾기
    div_tag = soup.find("div", class_="cts_atclst")
    if div_tag:
        articles = div_tag.find_all("li")  # <li> 태그 리스트 반환
        for article in articles:
            try:
                # 제목 추출
                title_tag = article.find("span", class_="tt2")
                title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
                titles.append(title)

                # 링크 추출
                link_tag = article.find("a")
                link = link_tag["href"] if link_tag and link_tag.has_attr("href") else "링크 없음"
                links.append(link)

                # 작성일자 추출
                date_tag = article.find("span", class_="pbdt")
                date = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
                dates.append(date)

            except Exception as e:
                print(f"개별 기사 처리 중 오류 발생: {e}")
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df.to_csv("이재명_연합뉴스_기사목록200.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.
